# Symbolic Verification of the paper "Extending Douglas–Rachford Splitting for Convex Optimization"

The following notebook serves as symbolic verification of the Lyapunov analysis of Algorithm 1 in the paper Extending Douglas–Rachford Splitting for Convex Optimization. The notation of this notebook follows that of the paper.

First, we will define the variables needed to run the sections in this notebook. Then, we will verify **(VR)** in Lemma 4.1. After that, we will verify **(LE)** in Proposition 4.1. Finally, we will verify the steps in Appendix A.

This notebook uses SymPy for symbolic calculations and IPython.display for a nicer rendering of mathematical expressions.

In [1]:
from sympy import *
from IPython.display import Math

To simplify the variable names in this notebook, we will drop the dependence of $k$ in the names. For instance, variables x_1, x_2 and z, should be interpreted as $x_1^k$, $x_2^k$ and $z^k$, while x_1plus, x_2plus and z_plus should be interpreted as $x_1^{k+1}$, $x_2^{k+1}$, and $z^{k+1}$, respectively, when comparing to the analysis of the paper. The variables f, f_plus and f_star represent $f(x_1^k)$, $f(x_1^{k+1})$ and $f(x^\star)$, respectively.

Throughout this notebook, products on the form $x_1*u_1$ should be interpreted as the inner product $\langle x_1, u_1\rangle$.

In [2]:
x_1, x_2, x_1plus, x_2plus, z, x_star, z_star = symbols('x_1 x_2 x_1plus x_2plus z x_star z_star')
f, f_plus, f_star = symbols('f, f_plus f_star')

The real numbers that will be used here are $\alpha, \tau$ and $\theta$. Note that any occurance of $\beta$ will be replaced by $\tau/\alpha$ in connection to the definition $\tau = \alpha/\beta$ in Definition 4.2. The reason for this is to make the code simpler, but also make some print outs in this notebook align with the expressions in the paper.




In [3]:
alpha, tau, theta = symbols('alpha tau theta', real = True)

By Algorithm 1, we have the update step $$z^{k+1} = z^k + \theta(x^k_2 - x^k_1)$$

In [4]:
z_plus = z + theta*(x_2 - x_1)

By Definition 4.1, we have the following expressions for $u_1^k$, $u_1^{k+1}$, $u_2^k$, $u_2^{k+1}$ and $u^\star$:
\begin{align*}
    u_1^k &= \frac{1}{\alpha}(z^k - x^k_1)\\
    u_1^{k+1} &= \frac{1}{\alpha}(z^{k+1} - x^{k+1}_1)\\
    u^k_2 &= \frac{1}{\beta}\left( \left( 1 + \frac{\beta}{\alpha}\right) x^k_1 - \frac{\beta}{\alpha}z^k - x^k_2\right)\\
    u^{k+1}_2 &= \frac{1}{\beta}\left( \left( 1 + \frac{\beta}{\alpha}\right) x^{k+1}_1 - \frac{\beta}{\alpha}z^{k+1} - x^{k+1}_2\right)\\
    u^\star &= \frac{1}{\alpha}(z^\star - x^\star)
\end{align*} which is the same as
\begin{align*}
    u_1^k &= \frac{1}{\alpha}(z^k - x^k_1)\\
    u_1^{k+1} &= \frac{1}{\alpha}(z^{k+1} - x^{k+1}_1)\\
    u^k_2 &= \frac{\tau}{\alpha}\left( \left( 1 + \tau\right) x^k_1 - \tau z^k - x^k_2\right)\\
    u^{k+1}_2 &= \frac{\tau}{\alpha}\left( \left( 1 + \tau\right) x^{k+1}_1 - \tau z^{k+1} - x^{k+1}_2\right)\\
    u^\star &= \frac{1}{\alpha}(z^\star - x^\star)
\end{align*}

In [5]:
u_1 = 1/alpha*(z - x_1)

u_1plus = 1/alpha*(z_plus - x_1plus)

u_2 = tau/alpha*((1 + 1/tau)*x_1 - 1/tau*z - x_2)

u_2plus = tau/alpha*((1 + 1/tau)*x_1plus - 1/tau*z_plus - x_2plus)

u_star = 1/alpha*(z_star - x_star)

We now define the Bregman-type distance
$$D_f(x, y, u) = f(x) - f(y) - \langle u, x - y \rangle$$ denoted by D(x, y, u) in the code. In this notebook, we will only work with the Bregman-type distance of $f$ evaluated at points $x^k_1, x^{k+1}_1$ or $x^\star$ and so the following definition satisfies our needs.

In [6]:
def D(x, y, u):
    f_eval = {
        x_1 : f,
        x_1plus : f_plus,
        x_star: f_star
    }
    return f_eval[x] - f_eval[y] - u*(x - y)

By Definition 4.2, we have the following expressions for $Q^{(i)}_k$, $V_k$, and $R_k$:
    \begin{align*}
        Q^{(1)}_k &=  \Bigl\lVert z^k-z^\star + \frac{\theta}{2}(x_2^k-x_1^k) + (\tau-1)(x_1^k-x^\star)\Bigr\rVert^2 + \frac{\theta(4\tau-\theta)}{4} \Bigl\lVert x_2^k-x_1^k \Bigr\rVert^2 + \tau(1-\tau) \Bigl\lVert x_1^k-x^\star \Bigr\rVert^2, \\
        V_k &= Q^{(1)}_k + 2\alpha(1 - \tau)D_f(x^k_1, x^\star, u^\star), \\
        R_k &= \theta(2\tau-\theta)\Bigl\lVert x_1^k-x_2^k \Bigr\rVert^2 + (1-\tau)\Bigl\lVert x_1^{k+1}-x_1^k\Bigr\rVert^2 + 2\alpha(1 - \tau)D_f(x^k_1, x^{k+1}_1, u^{k+1}_1)
    \end{align*}


In the code we will denote $Q^{(1)}_k$, $Q^{(1)}_{k+1}$, $V_k$, $V_{k+1}$, and $R_k$ by Q1, Q1_plus, V, V_plus, and R, respectively.

In [7]:
Q1 = (z - z_star + theta/2*(x_2 - x_1) + (tau - 1)*(x_1 - x_star))**2 \
    + theta*(4*tau - theta)/4 * (x_2 - x_1)**2 \
    + tau*(1 - tau)*(x_1 - x_star)**2

Q1_plus = (z_plus - z_star + theta/2*(x_2plus - x_1plus) + (tau - 1)*(x_1plus - x_star))**2 \
    + tau*(1 - tau)*(x_1plus - x_star)**2\
    + theta*(4*tau - theta)/4 * (x_2plus - x_1plus)**2

V = Q1 + 2*alpha*(1 - tau)*D(x_1, x_star, u_star)

V_plus = Q1_plus + 2*alpha*(1 - tau)*D(x_1plus, x_star, u_star)

R = theta*(2*tau - theta)*(x_1 - x_2)**2 \
    + (1 - tau)*(x_1plus - x_1)**2 \
    + 2*alpha*(1 - tau)*D(x_1, x_1plus, u_1plus)

By Lemma 4.1, we have the following expression for $Q^{(2)}_k$
    \begin{align*}
        Q^{(2)}_k &= \Bigl\lVert z^k-z^\star +\frac{\theta}{2}(x_2^k-x_1^k)\Bigr\rVert^2 +\frac{\theta(4\tau-\theta)}{4}\Bigl\lVert x_2^k-x_1^k +  \frac{2(\tau-1)}{4\tau-\theta}(x_1^k-x^\star)\Bigr\rVert^2 + \frac{\tau(\tau-1)(4-\theta)}{4\tau-\theta}\Bigl\lVert x_1^k-x^\star\Bigr\rVert^2\\
    \end{align*}
and the following right hand sides of **(VR)**
    \begin{align*}
        \bar{V}_k &= Q^{(2)}_k + 2\alpha(\tau - 1) D_f(x^\star, x^k_1, u^k_1), \\
        \bar{R}_k &= (\tau-1)\Bigl\lVert x_1^{k+1}-x_1^k + \theta(x_1^k-x_2^k)\Bigr\rVert^2 + \tau\theta(2-\theta)\Bigl\lVert x_1^k-x_2^k\Bigr\rVert^2 + 2\alpha(\tau - 1)D_f(x^{k+1}_1, x^{k}_1, u^{k}_1).
    \end{align*}
In the code we will denote $Q^{(2)}_k$, $\bar{V}_k$, and $\bar{R}_k$ by Q2, Vbar, and Rbar, respectively.

In [8]:
Q2 = (z - z_star + theta/2*(x_2 - x_1))**2 \
    + theta*(4*tau - theta)/4*(x_2 - x_1 + 2*(tau - 1)/(4*tau - theta)*(x_1 - x_star))**2 \
    + tau*(tau - 1)*(4 - theta)/(4*tau - theta)*(x_1 - x_star)**2

Vbar = Q2 + 2*alpha*(tau - 1)*D(x_star, x_1, u_1)

Rbar = (tau - 1)*(x_1plus - x_1 + theta*(x_1 - x_2))**2 \
    + tau*theta*(2 - theta)*(x_1 - x_2)**2 \
    + 2*alpha*(tau - 1)*D(x_1plus, x_1, u_1)

By Definition 4.3, we have the following expression for $I_k$:
$$
I_k= \langle u_1^k-u^\star, x_1^k- x^\star\rangle + \langle u_2^k+u^\star, x_2^k- x^\star\rangle+ \langle u_1^{k+1}-u^\star, x_1^{k+1}- x^\star\rangle+ \langle u_2^{k+1}+u^\star, x_2^{k+1}- x^\star\rangle.
$$ In the code, we will denote it by I.

In [9]:
I = (x_1 - x_star)*(u_1 - u_star) \
    + (x_1plus - x_star)*(u_1plus - u_star) \
    + (x_2 - x_star)*(u_2 + u_star) \
    + (x_2plus - x_star)*(u_2plus + u_star)

**Verification of equation (VR) in the proof of Lemma 4.1**

Now we verify that $V_k = V_k^{(2)}$ and $R_k = R_k^{(2)}$ for all $(\alpha, \beta, \theta)\in S^{(1)}\cup S^{(2)}$.

In [10]:
print(simplify(V - Vbar) == 0)

print(simplify(R - Rbar) == 0)

True
True


**Verification of equation (LE) in the proof of Lemma 4.1**

Now we verify that $V_{k+1} = V_k - R_k - \theta \alpha I_k$ for all $(\alpha, \beta, \theta)\in S^{(1)}\cup S^{(2)}$.

In [11]:
print(simplify(V_plus - V + R + alpha*theta*I) == 0)

True


**Verification of the steps in Appendix A**

First let us define the matrices
\begin{align*}
    Q_V^{(1)} &= \begin{bmatrix}
        \theta - \tau + 1 & -\frac{\theta(1+\tau)}{2} & 0 & 0 &\tau - 1 - \frac{\theta}{2}\\[6pt]
        * & \theta \tau & 0 & 0 &\frac{\theta}{2}\\[6pt]
        * & * & 0 & 0 & 0\\[6pt]
        * & * & * & 0 & 0\\[6pt]
        * & * & * & * & 1
        \end{bmatrix}, \\
    Q_V^{(2)} &= \begin{bmatrix}
         \theta + \tau - 1 & -\frac{\theta(1+\tau)}{2} &0 & 0 & - \frac{\theta}{2}\\[6pt]
        * & \theta \tau &0 & 0 &  \frac{\theta}{2}\\[6pt]
            * & * & 0 & 0 & 0\\[6pt]
        * & * & * & 0 & 0\\[6pt]
        * & * & * & * &1
        \end{bmatrix}, \\
    Q_R^{(1)} &= \begin{bmatrix}
        \theta(2\tau-\theta) - \tau + 1 & -\theta(2\tau - \theta) & -(1 - \tau) & 0 & 0\\[6pt]
        * & \theta(2\tau - \theta) & 0 & 0 & 0\\[6pt]
        * & * & 1 - \tau & 0 & 0\\[6pt]
        * & * & * & 0 & 0\\[6pt]
        * & * & * & * & 0
        \end{bmatrix}, \\
    Q_R^{(2)} &= \begin{bmatrix}
         \theta(2 - \theta) + \tau - 1 & \theta(\theta-1-\tau) & (\theta-1)(\tau-1) & 0 & 0\\[6pt]
        * & \theta(2\tau - \theta) & -\theta(\tau-1) & 0 & 0 \\[6pt]
        * & * & \tau - 1 & 0 & 0\\[6pt]
        * & * & * & 0 & 0\\[6pt]
        * & * & * & * & 0
    \end{bmatrix}, \\
\end{align*} which we will denote in the code by Q1_V, Q2_V, Q1_R, and Q2_R, respectively.

In [12]:
Q1_V = Matrix( [[theta - tau + 1, -theta/2*(1 + tau), 0, 0, tau - 1 - theta/2],
                [-theta/2*(1 + tau), theta*tau, 0, 0, theta/2],
                [0, 0, 0, 0, 0],
                [0, 0, 0, 0, 0],
                [tau - 1 - theta/2, theta/2, 0, 0, 1]])

Q2_V = Matrix( [[theta + tau - 1, -theta/2*(1 + tau), 0, 0, -theta/2],
                [-theta/2*(1+tau), theta*tau, 0,0,theta/2],
                [0, 0, 0, 0, 0],
                [0, 0, 0, 0, 0],
                [-theta/2, theta/2, 0, 0, 1]])

Q1_R = Matrix( [[theta*(2*tau - theta) - tau + 1, -theta*(2*tau - theta), -(1 - tau), 0, 0],
                [-theta*(2*tau - theta), theta*(2*tau - theta), 0, 0, 0],
                [-(1 - tau), 0, 1-tau, 0, 0],
                [0, 0, 0, 0, 0],
                [0, 0, 0, 0, 0]])

Q2_R = Matrix( [[theta*(2 - theta) + tau - 1, theta*(theta - 1 - tau), (theta - 1)*(tau - 1), 0, 0],
                [theta*(theta - 1- tau), theta*(2*tau - theta), -theta*(tau - 1), 0, 0],
                [(theta - 1)*(tau - 1), -theta*(tau - 1), tau - 1, 0, 0],
                [0, 0, 0, 0, 0],
                [0, 0, 0, 0, 0]])

Moreover, it will be helpful to define the vector
$$
v^k = \begin{bmatrix}
            x^k_1 - x^\star \\[6pt]
            x^k_2 - x^\star \\[6pt]
            x^{k+1}_1 - x^\star \\[6pt]
            x^{k+1}_2 - x^\star \\[6pt]
            z^k - z^\star
        \end{bmatrix}
$$ and we will denote it by v in the code.

Moreover, we will define a method that evaluates the quadratic form $\mathscr{Q}(Q, v^k)$ for any symmetric $5 \times 5$ matrix $Q$, which we will call Q_form.

In [13]:
v = Matrix([[x_1-x_star], [x_2-x_star], [x_1plus-x_star], [x_2plus-x_star], [z-z_star]])

def Q_form(Q):
    return (v.T @ Q @ v)[0]

We begin by verifying that


\begin{align*}
V_k &= \mathscr{Q}\left(Q_V^{(1)},
        v^k
        \right) + 2\alpha(1-\tau) D_f(x^k_1, x^\star, u^\star), \\
\bar{V}_k &= \mathscr{Q}\left(Q_V^{(2)},
        v^k
        \right) + 2\alpha(\tau - 1) D_f(x^\star, x^k_1, u^k_1), \\
R_k &= \mathscr{Q}\left(Q_R^{(1)},
        v^k
        \right) + 2\alpha(1-\tau)D_f(x_1^k, x_1^{k+1}, u_1^{k+1}), \\
\bar{R}_k &= \mathscr{Q}\left(Q_R^{(2)},
        v^k
        \right) + 2\alpha(\tau-1)D_f( x_1^{k+1}, x_1^k,u_1^{k}).
\end{align*} holds for all $(\alpha, \beta, \theta) \in S^{(1)} \cup S^{(2)}.$

In [14]:
print(simplify(V - Q_form(Q1_V) - 2*alpha*(1 - tau)*D(x_1, x_star, u_star)) == 0)

print(simplify(Vbar - Q_form(Q2_V) - 2*alpha*(tau - 1)*D(x_star, x_1, u_1)) == 0)

print(simplify(R - Q_form(Q1_R) - 2*alpha*(1 - tau)*D(x_1, x_1plus, u_1plus)) == 0)

print(simplify(Rbar - Q_form(Q2_R) - 2*alpha*(tau - 1)*D(x_1plus, x_1, u_1)) == 0)

True
True
True
True


With  \begin{equation}
M = \begin{bmatrix}
        0 & 0 & 1 & 0 & 0\\
        0 & 0 & 0 & 1 & 0\\
        0 & 0 & 0 & 0 & 0\\
        0 & 0 & 0 & 0 & 0\\
        -\theta & \theta & 0 & 0 & 1\\
\end{bmatrix}\end{equation} we now verify that
\begin{align*}
    M^\top Q_V^{(1)} M &= \begin{bmatrix}
        \theta^2 & - \theta^2 & -\theta(\tau-1-\frac{\theta}{2}) & - \frac{\theta^2}{2} & -\theta\\[6pt]
        * &  \theta^2 & \theta(\tau-1-\frac{\theta}{2}) &  \frac{\theta^2}{2} & \theta\\[6pt]
        * & * & 1 - \tau + \theta& - \frac{\theta(1+\tau)}{2} & \tau - 1 - \frac{\theta}{2}\\[6pt]
        * & * & * & \theta \tau & \frac{\theta}{2}\\[6pt]
        * & * & * & * & 1
    \end{bmatrix} \end{align*}

In [15]:
M = Matrix([[0, 0, 1, 0, 0],
            [0, 0, 0, 1, 0],
            [0, 0, 0, 0, 0],
            [0, 0, 0, 0, 0],
            [-theta, theta, 0, 0, 1]])

simplify(M.T@Q1_V@M)

Matrix([
[                    theta**2,                   -theta**2, theta*(-2*tau + theta + 2)/2,        -theta**2/2,            -theta],
[                   -theta**2,                    theta**2,  theta*(2*tau - theta - 2)/2,         theta**2/2,             theta],
[theta*(-2*tau + theta + 2)/2, theta*(2*tau - theta - 2)/2,             -tau + theta + 1, -theta*(tau + 1)/2, tau - theta/2 - 1],
[                 -theta**2/2,                  theta**2/2,           -theta*(tau + 1)/2,          tau*theta,           theta/2],
[                      -theta,                       theta,            tau - theta/2 - 1,            theta/2,                 1]])

and verify that
\begin{align*}
    M^\top Q_V^{(2)} M &= \begin{bmatrix}
    \theta^2 & -\theta^2 & \frac{\theta^2}{2} & - \frac{\theta^2}{2} & -\theta\\[6pt]
    * & \theta^2 & -\frac{\theta^2}{2} & \frac{\theta^2}{2} & \theta\\[6pt]
    * & * & \theta + \tau - 1 & -\frac{\theta(1+\tau)}{2} & -\frac{\theta}{2}\\[6pt]
    * & * & * & \theta \tau & \frac{\theta}{2}\\[6pt]
    * & * & * & * & 1
    \end{bmatrix}.
\end{align*}

In [16]:
simplify(M.T@Q2_V@M)

Matrix([
[   theta**2,   -theta**2,         theta**2/2,        -theta**2/2,   -theta],
[  -theta**2,    theta**2,        -theta**2/2,         theta**2/2,    theta],
[ theta**2/2, -theta**2/2,    tau + theta - 1, -theta*(tau + 1)/2, -theta/2],
[-theta**2/2,  theta**2/2, -theta*(tau + 1)/2,          tau*theta,  theta/2],
[     -theta,       theta,           -theta/2,            theta/2,        1]])

Moreover, we know verify that \begin{align*}
    V_{k+1} &= \mathscr{Q}(M^\top Q^{(1)}_V M, v^k) + 2\alpha (1 - \tau)D_f(x^{k+1}_1, x^\star, u^\star). \\
\end{align*}

In [17]:
print(simplify(V_plus - Q_form(M.T @ Q1_V @ M) - 2*alpha*(1 - tau)*D(x_1plus, x_star, u_star)) == 0)

True


We define the matrices
\begin{align*}
    Q_D &= \begin{bmatrix}
                -\theta & \tfrac{\theta}{2} & \tfrac{\theta - 1}{2} & 0 & \tfrac{1}{2} \\[6pt]
                * & 0 & -\tfrac{\theta}{2} & 0 & 0 \\[6pt]
                * & * & 1 & 0 & -\tfrac{1}{2} \\[6pt]
                * & * & * & 0 & 0 \\[6pt]
                * & * & * & * & 0
            \end{bmatrix}\\
\end{align*} represented by Q_D in the code.

In [18]:
Q_D = Matrix(  [[-theta, theta/2, (theta-1)/2, 0, S(1)/2],
                [theta/2, 0,-theta/2,0 , 0],
                [(theta-1)/2, -theta/2, 1, 0, -S(1)/2],
                [0, 0, 0, 0, 0],
                [S(1)/2, 0, -S(1)/2, 0, 0]])

We verify that
\begin{align*}
    D_f(x^k_1, x^\star, u^\star) - D_f(x^{k+1}_1, x^\star, u^\star) - D_f(x^k_1, x^{k+1}_1, u^{k+1}_1) &= \frac{1}{\alpha} \mathscr{Q}(Q_D, v^k). \\
\end{align*}

In [19]:
print(simplify(D(x_1, x_star, u_star) - D(x_1plus, x_star, u_star) - D(x_1, x_1plus, u_1plus) - 1/alpha*Q_form(Q_D)) == 0)

True


We define the matrix
\begin{align*}
    Q_I = \begin{bmatrix}
        -1 & \frac{1+\tau}{2} & -\frac{\theta}{2} & \frac{\theta}{2} & \frac{1}{2}\\[6pt]
        * & -\tau & \frac{\theta}{2} & -\frac{\theta}{2} & -\frac{1}{2}\\[6pt]
        * & * & -1 & \frac{1+\tau}{2} & \frac{1}{2}\\[6pt]
        * & * & * & -\tau & -\frac{1}{2}\\[6pt]
        * & * & * & * & 0
        \end{bmatrix}
\end{align*} represented by Q_I in the code.

In [20]:
Q_I = Matrix(  [[-S(1), (1+tau)/2, -theta/2, theta/2, S(1)/2],
                [(1+tau)/2, -tau, theta/2, -theta/2, -S(1)/2],
                [-theta/2, theta/2, -1, (1+tau)/2, S(1)/2],
                [theta/2, -theta/2, (1+tau)/2, -tau, -S(1)/2],
                [S(1)/2, -S(1)/2, S(1)/2, -S(1)/2, 0]])

We verify that $$\theta \alpha I_k = \theta \mathscr{Q}(Q_I, v^k).$$

In [21]:
print(simplify(theta*alpha*I - theta*Q_form(Q_I)) == 0)

True


Finally, we verify that  $$Q_V^{(1)}-M^\top Q_V^{(1)}M-Q_R^{(1)} + 2(1-\tau)Q_D = \theta Q_I$$ holds.

In [22]:
print(simplify(Q1_V - M.T@Q1_V@M - Q1_R + 2*(1-tau)*Q_D - theta*Q_I).is_zero_matrix)

True
